PoseEstimation flow:
============================

In [ ]:
# seeding

import pytorch_lightning as pl
import randomname

# get random version name before seeding
version = randomname.get_name()
seed = 22742

pl.seed_everything(seed, workers=True)

Model to use:
-------------

In [ ]:
from pedestrians_video_2_carla.data.carla.skeleton import CARLA_SKELETON
DATA_NODES = CARLA_SKELETON
INPUT_NODES = CARLA_SKELETON
OUTPUT_NODES = CARLA_SKELETON

In [ ]:
from pedestrians_video_2_carla.modules.pose_estimation.linear import Linear

# any model needs to inherit from `pedestrians_video_2_carla.modules.pose_estimation.pose_estimation.PoseEstimationModel`
model = Linear(
    input_nodes=INPUT_NODES,
    output_nodes=OUTPUT_NODES,
    # standard model params:
    movements_lr=0.0001,
    movements_enable_lr_scheduler=True,
    movements_scheduler_type="StepLR",
    movements_scheduler_gamma=0.333
)

In [ ]:
ckpt_path=None

Flow definition:
----------------

Also handles checkpoint loading if needed.

In [ ]:
# get flow model (LitPoseEstimationFlow)

from pedestrians_video_2_carla.modules.flow.pose_estimation import LitPoseEstimationFlow

if ckpt_path is not None:
    flow = LitPoseEstimationFlow.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        movements_model=model,
        input_nodes=INPUT_NODES,
        output_nodes=OUTPUT_NODES,
        loss_modes=["heatmaps"],
    )
else:
    flow = LitPoseEstimationFlow(
        movements_model=model,
        input_nodes=INPUT_NODES,
        output_nodes=OUTPUT_NODES,
        loss_modes=["heatmaps"],
    )

In [ ]:
# DataModule to use

from pedestrians_video_2_carla.data.carla.carla_recorded_video_datamodule import CarlaRecordedVideoDataModule
from pedestrians_video_2_carla.data.base.base_transforms import BaseTransforms

dm = CarlaRecordedVideoDataModule(
    batch_size=1,
    num_workers=0, # to simplify profiling/debugging
    clip_offset=5,
    clip_length=5,
    transforms=BaseTransforms.hips_neck_bbox,
    data_nodes=DATA_NODES,
    input_nodes=INPUT_NODES,
    needs_heatmaps=True,
    sigma=3,  # for heatmaps generation
    fast_dev_run=True
)

In [ ]:
# prepare data if needed - this will be done automatically if you use Trainer.fit(), but moved here explicitly for clarity

dm.prepare_data()

In [ ]:
# loggers and callbacks

import os

from torch.profiler import ProfilerActivity

from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, ModelSummary
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.profiler import PyTorchProfiler

from pedestrians_video_2_carla.loggers.pedestrian import PedestrianLogger
from pedestrians_video_2_carla.loggers.pedestrian.enums import PedestrianRenderers

import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = '/app/notebooks/flows/pose_estimation/profile.ipynb'

logger = WandbLogger(
    save_dir='/runs',
    name=version,
    version=version,
    project="pose_estimation",
    entity="carla-pedestrians",
    log_model=True,  # this will log models created by ModelCheckpoint,
    tags=["ipynb"],
)
log_dir = os.path.realpath(os.path.join(str(logger.experiment.dir), ".."))

pedestrian_logger = PedestrianLogger(
    save_dir=os.path.join(log_dir, "videos"),
    name=logger.name,
    version=logger.version,
    renderers=[PedestrianRenderers.source_videos, PedestrianRenderers.target_points, PedestrianRenderers.projection_points],
    source_videos_dir='/datasets/CARLA/WideCameraPedestrians',
    source_videos_overlay_bboxes=True,
    source_videos_overlay_skeletons=True,
    video_saving_frequency_reduction=0,  # set to 0 to disable (or empty renderers list)
    max_videos=1,
    input_nodes=INPUT_NODES,
    output_nodes=OUTPUT_NODES,
)
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(log_dir, "checkpoints"),
    monitor="val_loss/primary",
    mode="min",
    save_top_k=1,
)
lr_monitor = LearningRateMonitor(logging_interval="step")
model_summary = ModelSummary(max_depth=3)
profiler = PyTorchProfiler(
    export_to_chrome=True,
    record_module_names=True,
    profile_memory=True,
    activities=[ProfilerActivity.CPU],
    dirpath="profiler",
    filename="table.txt",
    row_limit=-1,
    sort_by_key="self_cpu_memory_usage",
)

In [ ]:
# actual trainer

dataset_fraction = 0.002 #0.001

trainer = pl.Trainer(
    log_every_n_steps=1,
    num_sanity_val_steps=8,
    check_val_every_n_epoch=1,
    limit_val_batches=dataset_fraction,
    limit_train_batches=dataset_fraction,
    max_epochs=1,
    deterministic=False,
    fast_dev_run=False,
    logger=[logger, pedestrian_logger],
    callbacks=[checkpoint_callback, lr_monitor, model_summary],
    profiler=profiler,
)

In [ ]:
# train! this will decode meta at the beginning with no visible output, it can take a few minutes

trainer.fit(model=flow, datamodule=dm, ckpt_path=ckpt_path)  # ckpt_path is needed to resume optimizer states etc.

In [ ]:
wandb.finish()